# Lesson 4 - Understanding User Intent

In this lesson, you will build the first agent of your multi-agent system, which will be the user intent agent.

You'll learn:

- how to give an agent a clear task
- how to define tools that are appropriate for the task
- how use state to save important information

Along the way, you will get experience with basic human-in-the-loop interaction

## 4.1. Agent Details

<img src="../Images/entire_solution.png" width="500">

The user intent agent is a goal-oriented, conversational agent that helps the user ideate on the kind of graph to build.
- Input: nothing
- Output: `approved_user_goal`, a dictionary pairing a kind of graph with a description of the purpose of the graph.
- Tools: `set_perceived_user_goal`, `approve_perceived_user_goal`

**Note**: Each lesson will focus on one part of the multi-agent system (as shown in the above image). The end-to-end solution will be available in this [repo](https://github.com/neo4j-contrib/agentic-kg). If the repo is empty, it means that the project is still in progress.

## 4.2. Setup

The usual import of needed libraries, loading of environment variables, and connection to Neo4j.

In [15]:
# Import necessary libraries

# Python packages
import os
import sys
sys.path.append('..')
import warnings
warnings.filterwarnings("ignore")
import logging
logging.basicConfig(level = logging.CRITICAL)
from typing import Optional, Dict, Any

# Graph Database for flow
from helpers.neo4j_for_adk import tool_success, tool_error, graphdb


# Agent caller from helper
from helpers.helper import make_agent_caller

# Google ADK libraries
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types
from google.adk.tools import ToolContext

print("Libraries imported")

Libraries imported


In [16]:
MODEL_GPT = "openai/gpt-4o"

llm = LiteLlm(model=MODEL_GPT)

# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, 
                                messages=[{"role": "user", "content": "Are you ready?"}], 
                                tools=[]))

print("\nOpenAI is ready for use.")

ModelResponse(id='chatcmpl-CENJQels5iMChvbanUiImZs97lQRM', created=1757542240, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_f33640a400', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Yes, I'm ready! How can I assist you today?", role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=13, prompt_tokens=27, total_tokens=40, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

OpenAI is ready for use.


## 4.3. Define the User Intent Agent

### 4.3.1 Agent Instructions

In [17]:
# define the role and goal for the user intent agent

agent_role_and_goal = """
    You are an expert at knowledge graph use cases. 
    Your primary goal is to help the user come up with a knowledge graph use case.
"""

In [18]:
# give the agent some hints about what to say

agent_conversational_hints = """
    If the user is unsure what to do, make some suggestions based on classic use cases like:
    - social network involving friends, family, or professional relationships
    - logistics network with suppliers, customers, and partners
    - recommendation system with customers, products, and purchase patterns
    - fraud detection over multiple accounts with suspicious patterns of transactions
    - pop-culture graphs with movies, books, or music
"""

In [19]:
# describe what the output should look like
agent_output_definition = """
    A user goal has two components:
    - kind_of_graph: at most 3 words describing the graph, for example "social network" or "USA freight logistics"
    - description: a few sentences about the intention of the graph, for example "A dynamic routing and delivery system for cargo." or "Analysis of product dependencies and supplier alternatives."
"""

In [20]:
# specify the steps the agent should follow
agent_chain_of_thought_directions = """
    Think carefully and collaborate with the user:
    1. Understand the user's goal, which is a kind_of_graph with description
    2. Ask clarifying questions as needed
    3. When you think you understand their goal, use the 'set_perceived_user_goal' tool to record your perception
    4. Present the perceived user goal to the user for confirmation
    5. If the user agrees, use the 'approve_perceived_user_goal' tool to approve the user goal. This will save the goal in state under the 'approved_user_goal' key.
"""


In [21]:
# combine all the instruction components into one complete instruction...
complete_agent_instruction = f"""
{agent_role_and_goal}
{agent_conversational_hints}
{agent_output_definition}
{agent_chain_of_thought_directions}
"""

print(complete_agent_instruction)



    You are an expert at knowledge graph use cases. 
    Your primary goal is to help the user come up with a knowledge graph use case.


    If the user is unsure what to do, make some suggestions based on classic use cases like:
    - social network involving friends, family, or professional relationships
    - logistics network with suppliers, customers, and partners
    - recommendation system with customers, products, and purchase patterns
    - fraud detection over multiple accounts with suspicious patterns of transactions
    - pop-culture graphs with movies, books, or music


    A user goal has two components:
    - kind_of_graph: at most 3 words describing the graph, for example "social network" or "USA freight logistics"
    - description: a few sentences about the intention of the graph, for example "A dynamic routing and delivery system for cargo." or "Analysis of product dependencies and supplier alternatives."


    Think carefully and collaborate with the user:
    1.

### 4.3.2 Tool Definition

In [22]:
# Tool: Set Percieved User Goal
# to encourage collaboration with the user, the first tool only sets the perceived user goal

PERCEIVED_USER_GOAL = "perceived_user_goal"

def set_perceived_user_goal(kind_of_graph: str, graph_description: str, tool_context: ToolContext):
    """Sets the perceived user's goal, including the kind of graph and its description.
    
    Args:
        kind_of_graph: 2-3 word definition of the kind of graph, for example "recent US patents"
        graph_description: a single paragraph description of the graph, summarizing the user's intent
    """
    user_goal_data = {
        "kind_of_graph": kind_of_graph,
        "graph_description": graph_description
        }
    tool_context.state[PERCEIVED_USER_GOAL] = user_goal_data
    return tool_success(PERCEIVED_USER_GOAL, user_goal_data)



In [23]:
# Tool : Approve the perceived user goal
# approval from the user should trigger a call to this tool

APPROVED_USER_GOAL = "approved_user_goal"

def approve_perceived_user_goal(tool_context:ToolContext):
    """Upon approval from user, will record the perceived user goal as the approved user goal.
    
    Only call this tool if the user has explicitly approved the perceived user goal.
    """
    # Trust, but verify. 
    # Require that the perceived goal was set before approving it. 
    # Notice the tool error helps the agent take    
    if PERCEIVED_USER_GOAL not in tool_context.state:
        return tool_error("perceived_user_goal not set. Set perceived user goal first, or ask clarifying questions if you are unsure")
    
    tool_context.state[APPROVED_USER_GOAL] = tool_context.state[PERCEIVED_USER_GOAL]
    return tool_success(APPROVED_USER_GOAL, tool_context.state[PERCEIVED_USER_GOAL])


In [24]:
# add the tools to a list
user_intent_agent_tools = [set_perceived_user_goal, approve_perceived_user_goal]

### 4.3.3 Agent Definition

In [25]:
# Construct the agent
user_intent_agent = Agent(
    name = "user_intent_agent_v1", # A unique versioned name
    model = llm, # Defined earlier
    description = "Helps the user ideate on a knowledge graph use case.", # used for delegation
    instruction = complete_agent_instruction, # the complete instructions you composed earlier
    tools = user_intent_agent_tools, # list of tools
)

print(f"Agent '{user_intent_agent.name}' created")

Agent 'user_intent_agent_v1' created


## 4.4. Interact with the Agent

In [26]:
 # use a helper to create an agent execution environment
 user_intent_caller = await make_agent_caller(user_intent_agent)

In [27]:
# Run the Initial Conversation

session_start = await user_intent_caller.get_session()
print(f"Session Start: {session_start.state}")

# Async function to await for each conversation
async def run_conversation():
    # start things off by describing your goal
    await user_intent_caller.call(
        """I'd like a bill of materials graph (BOM graph) which includes all levels from suppliers to finished product, 
    which can support root-cause analysis."""
    )

    if PERCEIVED_USER_GOAL not in session_start.state:
        # the LLM may have asked a clarifying question. Offer some more details
        await user_intent_caller.call("""I'm concerned about possible manufacturing or supplier issues.""")  
    
    # Presume approval
    await user_intent_caller.call("Approve that goal", True)


# Run the conversation
await run_conversation()

session_end = await user_intent_caller.get_session()




Session Start: {}

>>> User Query: I'd like a bill of materials graph (BOM graph) which includes all levels from suppliers to finished product, 
    which can support root-cause analysis.
<<< Agent Response: A Bill of Materials (BOM) graph focusing on entire supply chains from suppliers to finished products is an excellent use case. You want to include all levels and ensure it supports root-cause analysis. This typically requires mapping out components, sub-components, and hierarchy in manufacturing, identifying how parts are linked, and tracing suppliers and manufacturing processes.  

Before confirming this as your goal, could I ask a few clarifying questions?
1. Are there specific industries or types of products you're targeting?
2. Do you want to include any historical data or focus on real-time updates?
3. Would you like to integrate external data sources, such as market or economic trends, to aid in the analysis?
4. Are there specific metrics or types of analysis you're intereste

Take a close look at the session state.

- session state starts empty, as you'd expect since we did not initialize it
- after the conversation, there are two values in session state: `perceived_user_goal` and `approved_user_goal`
- this duplication is intentional, separating "working memory" from work specification
- subsequent steps in the workflow should only use approved work specifications
- in production, work specifications should be persisted to enable tracing and reproducibility

In [28]:
session_start.state

{}

## 4.5. Optional - Sequence diagram illustrating the workflow of  "User Intent Agent"  

<img src="../Images/user_intent_diag.png"  width=600> 